# Categorical Boost
#### cv score :  0.5639017013232513 (5 folds, LB 0.5609)
#### cv score :  0.5639017013232513 (5 folds, find rare, LB 0.5575)
#### cv score :  0.571281663516068 ... (LB 0.555) onehotなし、

In [1]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
import sklearn
from Functions import prepro
import warnings
warnings.filterwarnings('ignore')

import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore')
from catboost import Pool, CatBoostClassifier, CatBoost
import matplotlib.pyplot as plt
import seaborn as sns


FEATURE_SELECTION = False
tgt_encode = False

random.seed(0)

pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 1000)

%matplotlib inline

In [2]:
train_df = pd.read_csv("../../data/Processed/train2.csv")
test_df = pd.read_csv("../../data/Processed/test2.csv")
print(train_df.shape)
print(test_df.shape)

(66125, 304)
(28340, 303)


In [3]:
y = train_df["y"].values
train_df = train_df.drop("y", axis=1)

train_df["A1-level"] = train_df["A1-level"].astype(float)
test_df["A1-level"] = test_df["A1-level"].astype(float)
train_df["B1-level"] = train_df["B1-level"].astype(float)
test_df["B1-level"] = test_df["B1-level"].astype(float)

In [4]:
# add disconnection info

train_df = prepro.add_disconnection(train_df)
test_df = prepro.add_disconnection(test_df)

In [ ]:
# add info about numeric column
# 武器データの中のnum_colsを持つ特徴について、チーム内の統計量を計算、levelは結構効いてる

# num_cols = ["level", "range", "rapid", "atack", "ink-sub", "fav-main", "good-special"]

num_cols = [
    "level", "range-main", "range-bullet-main", "distant-range_sub", 
    "rapid", "atack", "ink-sub", "fav-main", "good-special", "DPS", "kill_time_ika-main",
    "front_gap_human-main", "rensya_frame-main", "saidai_damege-main", "damage_min-sub", 
    "damage_max-sub", "install_num-sub", "good-sub", "damage_max-special", 
    "duration-special", "good-special", "direct_rad-special", "distant_rad-special"
]

cols = [col for col in train_df.columns if "A1" in col or "A2" in col or "A3" in col or "A4" in col or 
        "B1" in col or "B2" in col or "B3" in col or "B4" in col]
drop_cols = []
for col1 in cols:
    f = True
    for col2 in num_cols:
        if col2 in col1:
            f = False
    if f and train_df[col1].dtype in [int, float]:
        drop_cols.append(col1)
        
train_df = train_df.drop(columns=drop_cols)
test_df = test_df.drop(columns=drop_cols)
        
    

train_df = prepro.add_numeric_info(train_df, num_cols)
test_df = prepro.add_numeric_info(test_df, num_cols)



# 欠損値埋め先にやろう
欠損値がある列

rank, weapon, level ⇨　回線切れ or ナワバリ

weaponの欠損に並んでそれに関係ある列、A3, A4, B3, B4 ⇨ 回線切れ

level, weaponが消えていたら回線切れ

In [ ]:
# rankの欠損値を埋める
train_df, test_df = prepro.fillna_rank(train_df, test_df)

#そのほかの欠損値を埋める
train_df, test_df = prepro.fillna(train_df, test_df)

In [ ]:
print(train_df.isnull().sum().sum())
print(test_df.isnull().sum().sum())

# 特徴量エンジニアリング

In [ ]:
# count reskin, by mode
print("reskin")
train_df, test_df = prepro.count_reskin(train_df, test_df)
train_df, test_df = prepro.count_reskin_by_mode(train_df, test_df)

# count mainweapon, by mode
print("mainweapon")
train_df, test_df = prepro.count_mainweapon(train_df, test_df)
train_df, test_df = prepro.count_mainweapon_by_mode(train_df, test_df)

# count subweapon, by mode
print("subweapon")
train_df, test_df = prepro.count_subweapon(train_df, test_df)
train_df, test_df = prepro.count_subweapon_by_mode(train_df, test_df)

# count special, by mode
print("special")
train_df, test_df = prepro.count_special(train_df, test_df)
train_df, test_df = prepro.count_special_by_mode(train_df, test_df)


#identify A1
train_df, test_df = prepro.identify_A1(train_df, test_df)


# 水増し, A1も統計量に含めた特徴を作る場合は水ましより先にやる
print("mizumashi")
train_df, y = prepro.mizumashi(train_df, y)

# is_nawabari
train_df, test_df = prepro.is_nawabari(train_df, test_df)

# match rank、単体で意味なし
train_df, test_df = prepro.match_rank(train_df, test_df)

# rankを二列に分ける
train_df, test_df = prepro.ranker(train_df, test_df)




In [ ]:
# add team info、メインはなくてもいい

train_df,  test_df = prepro.addTeamInfo(train_df, test_df, cols=["special", "subweapon", "category1", "category2", "mainweapon"])
#train_df,  test_df = prepro.addTeamInfo(train_df, test_df, cols=["special", "subweapon", "category1", "category2"])

In [ ]:
# categorize team , 良い, メインはカテゴリ数が多すぎてやめた方がいい

categorize_col = ["category1", "category2", "subweapon", "special", "mainweapon"]
#categorize_col = ["category1", "category2", "subweapon", "special"]
for col in categorize_col:
    print(col)
    train_df, test_df = prepro.categorize_team(train_df, test_df, col)

In [ ]:
# レアなカテゴリをまとめる、しない方が上がった…

#train_df, test_df = prepro.find_rare(train_df, test_df, "team-category1")
#train_df, test_df = prepro.find_rare(train_df, test_df, "team-category2")
#train_df, test_df = prepro.find_rare(train_df, test_df, "team-subweapon")
#train_df, test_df = prepro.find_rare(train_df, test_df, "team-special")

In [ ]:
# product categorical feature
train_df, test_df = prepro.prod(train_df, test_df, "mode", "stage")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-category1-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-category1-B")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-category2-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-category2-B")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-mainweapon-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-mainweapon-B")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-subweapon-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-subweapon-B")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-special-A")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "team-special-B")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "match_rank")

train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-category1-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-category1-B")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-category2-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-category2-B")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-mainweapon-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-mainweapon-B")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-subweapon-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-subweapon-B")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-special-A")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "team-special-B")
train_df, test_df = prepro.prod(train_df, test_df, "stage", "match_rank")

"""
train_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-A1")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-A2")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-A3")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-A4")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-B1")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-B2")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-B3")
train_df, test_df = prepro.prod(train_df, test_df, "mode", "reskin-B4")"""

In [ ]:
"""#one hot 落とす
drop_cols = [col for col in train_df.columns if "onehot" in col]
train_df.drop(columns=drop_cols, inplace=True)
test_df.drop(columns=drop_cols, inplace=True)"""

In [ ]:
# make input

drop_cols = [
    "id", "lobby", "lobby-mode",  "period", "game-ver", 
    "A1-weapon", "A2-weapon", "A3-weapon", "A4-weapon", 
    "B1-weapon", "B2-weapon", "B3-weapon", "B4-weapon",
    "A1-reskin", "A2-reskin", "A3-reskin", "A4-reskin", 
    "B1-reskin", "B2-reskin", "B3-reskin", "B4-reskin",
    "category1-A1", "category1-A2", "category1-A3", "category1-A4", 
    "category1-B1", "category1-B2", "category1-B3", "category1-B4",
    "category2-A1", "category2-A2", "category2-A3", "category2-A4", 
    "category2-B1", "category2-B2", "category2-B3", "category2-B4",
    "mainweapon-A1", "mainweapon-A2", "mainweapon-A3", "mainweapon-A4", 
    "mainweapon-B1", "mainweapon-B2", "mainweapon-B3", "mainweapon-B4",
    "subweapon-A1", "subweapon-A2", "subweapon-A3", "subweapon-A4", 
    "subweapon-B1", "subweapon-B2", "subweapon-B3", "subweapon-B4",
    "special-A1", "special-A2", "special-A3", "special-A4", 
    "special-B1", "special-B2", "special-B3", "special-B4",
]

X, test_X = prepro.make_input(train_df, test_df, drop_cols, categorical_encode=False, scaler=False, verbose=False)

categorical_features_indices = np.where(X.dtypes == "object")[0]

# 学習

In [ ]:
# 全データを5つに分割
random.seed(random.randint(0, 10000))
SIZE = X.shape[0]
K = 5
cat_tgtenc_cols = [
    "team-category1-A", "team-category1-B", "team-category2-A", "team-category2-B",
    "team-subweapon-A", "team-subweapon-B", "team-special-A", "team-special-B"
]
cat_tgtenc_cols = []

#folds = prepro.make_stratified_kfolds(X, y, K, shuffle=True)
folds = prepro.make_stratified_kfolds(X, X["mode"].astype(str) + y.astype(str), K, shuffle=True, random_state=random.randint(0, 10000))


print(len(folds))
for i, fold in enumerate(folds):
    print("fold ", i+1, " size is ", len(fold))
    
    
if SIZE != len(set(sum(folds, []))):
    print("error is occuring in spliting")
else :
    print("successfully split")

In [ ]:
params = {
    "loss_function" : "Logloss",
    "eval_metric" : "Logloss",
    "iterations":2000,
    "learning_rate" : 0.05,
    "use_best_model": True,
    "random_seed":random.randint(0, 100000),

    

}

THRESHOLD = 0.50
models = []
cv_scores = []
temp = 0
train_pred = []
valid_ys = []

all_indices = sum(folds, [])
for i in range(K):
    print("======================== fold {} ========================".format(i+1))
    valid_indices = folds[i]
    train_indices = list(set(all_indices) - set(valid_indices))
    # print("train ", len(train_indices), " , valid ", len(valid_indices))
    

    train_X = X.iloc[train_indices]
    try:
        train_y = y.iloc[train_indices]
    except:
        train_y = y[train_indices]
    valid_X = X.iloc[valid_indices]
    try:
        valid_y = y.iloc[valid_indices]
    except:
        valid_y = y[valid_indices]
    
    
    
    
    train_data = Pool(train_X, train_y, cat_features=categorical_features_indices)
    valid_data = Pool(valid_X, valid_y, cat_features=categorical_features_indices)
    
    model = CatBoostClassifier(**params)

    
    model.fit(
        train_data,
        eval_set=valid_data,
        early_stopping_rounds=50,
        verbose=100,
        use_best_model=True,
    )
    
    pred = model.predict(valid_X, prediction_type='Probability')[:,1]
    train_pred.append(pred)
    pred = np.where(pred < THRESHOLD, 0, 1)

    temp += np.sum(pred) 
    
    score = accuracy_score(pred, valid_y)
    
    models.append(model)
    cv_scores.append(score)
    
    
print("cv score : ", np.mean(cv_scores))    
print("cv ratio : ", temp / SIZE)

In [ ]:
preds = []
for i in range(K):
    model = models[i]
    pred = model.predict(test_X, prediction_type='Probability')[:,1]
    preds.append(pred)
    print(np.sum(pred) / pred.shape[0])
    
preds = np.array(preds)
preds = np.mean(preds, axis=0)
print(np.sum(preds) / preds.shape[0])


temp = pd.DataFrame({"pred":pred})
temp.hist(bins=80)

In [ ]:
preds = np.where(preds < THRESHOLD, 0, 1)
print(np.sum(preds) / preds.shape[0])

submit_df = pd.DataFrame({'y': preds})
submit_df.index.name = 'id'
submit_df.to_csv('../Submissions/submission_cat_6_{}.csv'.format(K))

# モデル解釈

In [ ]:
importance = pd.DataFrame(
    (models[0].get_feature_importance() + 
    models[1].get_feature_importance() + 
    models[2].get_feature_importance() + 
    models[3].get_feature_importance() + 
    models[4].get_feature_importance())/5
    , index=X.columns, columns=['importance']).sort_values('importance', ascending=False)
display(importance[:120])

In [ ]:
indices = [index for index in importance.index if "team" in index]
importance.loc[indices]

In [ ]:
train_pred

In [ ]:
train_df[train_df["disconnection-A"] != 0][["disconnection-A", "disconnection-B", "y", "pred"]]

In [ ]:
train_df["pred"] = 0
train_df["y"] = y
for i in range(K):
    train_df["pred"].iloc[folds[i]] = np.where(train_pred[i] < THRESHOLD, 0, 1)
    #train_df["pred"].iloc[folds[i]] = train_pred[i]
for mode in train_df["mode"].unique():
    rate = train_df[train_df["mode"] == mode][train_df["pred"] == train_df["y"]].shape[0] / train_df[train_df["mode"] == mode].shape[0]
    print("{} : {}".format(mode, rate))
print(train_df[train_df["pred"] == train_df["y"]].shape[0] / train_df.shape[0])

In [ ]:
result_df = train_df[train_df["y"] != train_df["pred"]]

In [ ]:
print(result_df[result_df["mode"] == "nawabari"].shape[0]/result_df.shape[0])
print(result_df[result_df["mode"] == "hoko"].shape[0]/result_df.shape[0])
print(result_df[result_df["mode"] == "asari"].shape[0]/result_df.shape[0])
print(result_df[result_df["mode"] == "area"].shape[0]/result_df.shape[0])
print(result_df[result_df["mode"] == "yagura"].shape[0]/result_df.shape[0])

In [ ]:
result_df[result_df["mode"] == "nawabari"]

In [ ]:
preds

In [ ]:
def plot_feature_importance(importance,names,model_type, MAXSIZE=120):

    #Create arrays from feature importance and feature names
    feature_importance = np.array(importance)[:MAXSIZE]
    feature_names = np.array(names)[:MAXSIZE]

    #Create a DataFrame using a Dictionary
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)

    #Sort the DataFrame in order decreasing feature importance
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

    #Define size of bar plot
    plt.figure(figsize=(20,16))
    #Plot Searborn bar chart
    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
    #Add chart labels
    plt.title(model_type + 'FEATURE IMPORTANCE')
    plt.xlabel('FEATURE IMPORTANCE')
    plt.ylabel('FEATURE NAMES')

In [ ]:
plot_feature_importance(importance.values.reshape(-1,), importance.index, "Cat Boost ", MAXSIZE=40)

In [ ]:
train_df

In [ ]:
v = train_df["team-subweapon-A"].value_counts().add(
    train_df["team-subweapon-B"].value_counts().add(
        test_df["team-subweapon-A"].value_counts().add(
            test_df["team-subweapon-A"].value_counts(), fill_value=0
        ), fill_value=0
    ), fill_value=0
).sort_values(ascending=False)

In [ ]:
x = np.arange(len(v))
plt.bar(x, v)
plt.show()

In [ ]:
def find_rare(df1, df2, col, threshold=4):
    v_tra = df1[col + "-A"].value_counts()
    v_trb = df1[col + "-B"].value_counts()
    v_tea = df2[col + "-A"].value_counts()
    v_teb = df2[col + "-B"].value_counts()
    
    v_counts = v_tra.add(v_trb, fill_value=0)
    v_counts = v_counts.add(v_tea, fill_value=0)
    v_counts = v_counts.add(v_teb, fill_value=0)
    
    #threshold = v_counts.iloc[int(v_counts.shape[0]*0.8)]

    tra = df1[col + "-A"].unique()
    trb = df1[col + "-B"].unique()
    tea = df2[col + "-A"].unique()
    teb = df2[col + "-B"].unique()

    not_appeared = []
    print("fin count")
    for item in v_counts.index:
        if item not in tra or item not in trb or item not in tea or item not in teb or v_tra.loc[item] < 5 or v_trb.loc[item] < 5:
            not_appeared.append(item)

    print("fin find rare", len(not_appeared))
    df1[col + "-A"] = df1[col + "-A"].map(lambda x: "rare" if x in not_appeared else x)
    df1[col + "-B"] = df1[col + "-B"].map(lambda x: "rare" if x in not_appeared else x)
    df2[col + "-A"] = df2[col + "-A"].map(lambda x: "rare" if x in not_appeared else x)
    df2[col + "-B"] = df2[col + "-B"].map(lambda x: "rare" if x in not_appeared else x)

    return df1, df2

In [ ]:
train_df, test_df = find_rare(train_df, test_df, "team-special") #473 sub

In [ ]:
test_df["team-special-A"].value_counts().sort_values(ascending=False)

In [ ]:
train_df